# Big Data Mining: HW#2

## Tri Wanda Septian (106998406), Anggara Aji Saputra (106998412)

### Input data
### Data: 
- [News Popularity in Multiple Social Media Platforms dataset] from UCI Machine Learning Repository
- About 100,000 news items on four different topics: economy, microsoft, Obama, and Palestine, during November 2015 and July 2016
- Available at<br>
https://archive.ics.uci.edu/ml/datasets/News+Popularity+in+Multiple+Social+Media+Platforms 

### Format:
- News data: 1 CSV file
- Social feedback data: CSV files for 4 topics on 3 platforms

### Attributes of News Data
- IDLink (numeric): Unique identifier of news items 
- Title (string): Title of the news item according to the official media sources 
- Headline (string): Headline of the news item according to the official media sources 
- Source (string): Original news outlet that published the news item 
- Topic (string): Query topic used to obtain the items in the official media sources 
- PublishDate (timestamp): Date and time of the news items' publication 
- SentimentTitle (numeric): Sentiment score of the text in the news items' title 
- SentimentHeadline (numeric): Sentiment score of the text in the news items' headline 
- Facebook (numeric): Final value of the news items' popularity according to the social media source Facebook 
- GooglePlus (numeric): Final value of the news items' popularity according to the social media source Google+ 
- LinkedIn (numeric): Final value of the news items' popularity according to the social media source LinkedIn 

### Attributes of Social Feedback Data
- IDLink (numeric): Unique identifier of news items 
- TS1 (numeric): Level of popularity in time slice 1 (0-20 minutes upon publication) 
- TS2 (numeric): Level of popularity in time slice 2 (20-40 minutes upon publication) 
- TS... (numeric): Level of popularity in time slice ... 
- TS144 (numeric): Final level of popularity after 2 days upon publication

### Task Description
### Subtasks
- (30pt) (1) In news data, count the words in two fields: ‘Title’ and ‘Headline’ respectively, and list the most frequent words according to the term frequency in descending order, in total, per day, and per topic, respectively
- (20pt) (2) In social feedback data, calculate the average popularity of each news by hour, and by day, respectively (for each platform)
- (20pt) (3) In news data, calculate the sum and average sentiment score of each topic, respectively
- (30pt) (4) From subtask (1), for the top-100 frequent words per topic in titles and headlines, calculate their co-occurrence matrices (100x100), respectively. Each entry in the matrix will contain the co-occurrence frequency in all news titles and headlines, respectively

### Output Format
- (1) 6 sorted lists of top-frequent words: {in total, per day, per topic}{for titles, headlines}
    - Each line: <word> <count>
    - For sorted lists per day/per topic, you can also separate them into individual lists by day/by topic 
        - That will make more numbers of sorted lists: 60*2 lists by day, and 4*2 by topic. 
- (2) 6 files: {by hour, by day} {3 platforms}
    - Each line: <avg popularity>
- (3) 8 values: {sum, avg} {4 topics}
- (4) 8 100x100 matrices: {title, headline} {4 topics}
    - Each entry mij in the matrix: the co-occurrence frequency of wi and wj

## Specification

In [67]:
import os
import sys

### Memory (RAM)

In [69]:
%cat /proc/meminfo

MemTotal:        8168740 kB
MemFree:          189660 kB
MemAvailable:     661772 kB
Buffers:           94304 kB
Cached:           713748 kB
SwapCached:       128768 kB
Active:          6040500 kB
Inactive:        1543152 kB
Active(anon):    5674432 kB
Inactive(anon):  1269964 kB
Active(file):     366068 kB
Inactive(file):   273188 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:       8385532 kB
SwapFree:        7032116 kB
Dirty:               296 kB
Writeback:             0 kB
AnonPages:       6767680 kB
Mapped:           354944 kB
Shmem:            168792 kB
Slab:             200824 kB
SReclaimable:     108528 kB
SUnreclaim:        92296 kB
KernelStack:       30400 kB
PageTables:        74496 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    12469900 kB
Committed_AS:   18385036 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
HardwareCorrupted:     0 kB
AnonHugePages:   

### CPU

In [70]:
%cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU E5-2620 v3 @ 2.40GHz
stepping	: 2
microcode	: 0x35
cpu MHz		: 2397.223
cache size	: 15360 KB
physical id	: 0
siblings	: 1
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts mmx fxsr sse sse2 ss syscall nx pdpe1gb rdtscp lm constant_tsc arch_perfmon pebs bts nopl xtopology tsc_reliable nonstop_tsc cpuid pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand hypervisor lahf_lm abm cpuid_fault invpcid_single pti retpoline fsgsbase tsc_adjust bmi1 avx2 smep bmi2 invpcid xsaveopt arat
bugs		: cpu_meltdown spectre_v1 spectre_v2
bogomips	: 4794.44
clflush size	: 64
cache_alignment	: 64
address sizes	: 43 bits physical, 48 bits virtual
power management:

processor	: 1
vendor_id	: Gen

### Operating System

In [71]:
%cat /etc/lsb-release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=16.04
DISTRIB_CODENAME=xenial
DISTRIB_DESCRIPTION="Ubuntu 16.04.4 LTS"


### Spark Setup 

pyspark.sql.SparkSession Main entry point for DataFrame and SQL functionality.

In [1]:
from pyspark.sql import SparkSession

Builder for SparkSession.

appName(name)

Sets a name for the application, which will be shown in the Spark web UI.
If no application name is set, a randomly generated name will be used.
Parameters:	name – an application name

getOrCreate()

Gets an existing SparkSession or, if there is no existing one, creates a new one based on the options set in this builder.

This method first checks whether there is a valid global default SparkSession, and if yes, return that one. If no valid global default SparkSession exists, the method creates a new SparkSession and assigns the newly created SparkSession as the global default.

In [2]:
spark = SparkSession.builder.appName("HW#2").getOrCreate()

In [6]:
spark

### Load News_Final.csv files dataset from hdfs
hdfs url : hdfs://tri-virtual-machine:9000

In [3]:
data = spark.read.csv("hdfs://tri-virtual-machine:9000/user/tri/News_Final.csv", header = True, inferSchema = True)

Print the schema in a tree format

In [4]:
data.printSchema()

root
 |-- IDLink: decimal(10,0) (nullable = true)
 |-- Title: string (nullable = true)
 |-- Headline: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Topic: string (nullable = true)
 |-- PublishDate: string (nullable = true)
 |-- SentimentTitle: string (nullable = true)
 |-- SentimentHeadline: string (nullable = true)
 |-- Facebook: string (nullable = true)
 |-- GooglePlus: string (nullable = true)
 |-- LinkedIn: string (nullable = true)



### Show the dataset

In [7]:
data.show()

+------+--------------------+--------------------+--------------------+---------+-------------------+-------------------+--------------------+--------+----------+--------+
|IDLink|               Title|            Headline|              Source|    Topic|        PublishDate|     SentimentTitle|   SentimentHeadline|Facebook|GooglePlus|LinkedIn|
+------+--------------------+--------------------+--------------------+---------+-------------------+-------------------+--------------------+--------+----------+--------+
| 99248|Obama Lays Wreath...|Obama Lays Wreath...|           USA TODAY|    obama|2002-04-02 00:00:00|                  0| -0.0533001790889026|      -1|        -1|      -1|
| 10423|A Look at the Hea...|Tim Haywood, inve...|           Bloomberg|  economy|2008-09-20 00:00:00|  0.208333333333333|  -0.156385810542806|      -1|        -1|      -1|
| 18828|Nouriel Roubini: ...|Nouriel Roubini, ...|           Bloomberg|  economy|2012-01-28 00:00:00| -0.425210032135381|   0.13975424859373

### Show "Title" and "Headline" from dataset

select(*cols)

Projects a set of expressions and returns a new DataFrame.

Parameters:	cols – list of column names (string) or expressions (Column). If one of the column names is ‘*’, that column is expanded to include all columns in the current DataFrame

In [8]:
data.select("Title","Headline").show()

+--------------------+--------------------+
|               Title|            Headline|
+--------------------+--------------------+
|Obama Lays Wreath...|Obama Lays Wreath...|
|A Look at the Hea...|Tim Haywood, inve...|
|Nouriel Roubini: ...|Nouriel Roubini, ...|
|Finland GDP Expan...|Finland's economy...|
|Tourism, govt spe...|Tourism and publi...|
|Intellitec Soluti...|Over 100 attendee...|
| Monday, 29 Feb 2016|RAMALLAH, Februar...|
|Obama, stars pay ...|First lady Michel...|
|Fire claims more ...|A Hancock County ...|
|Microsoft's new W...|New Delhi, Feb.29...|
|Microsoft Project...|Microsoft may hav...|
|Microsoft sneaks ...|The platform batt...|
|Greek economy gro...|Greece's economy ...|
|Big data and the ...|Big data analytic...|
|HoloLens dev edit...|Microsoft’s AR he...|
|Microsoft Word fo...|What is A + B? We...|
|Microsoft Band 2 ...|The Microsoft Ban...|
|Microsoft prepare...|It seems that Mic...|
|Greek economy shr...|Greece's economy ...|
|Sweden's economy ...|Sweden's e

### Count by "Title"

groupBy(*cols)

Groups the DataFrame using the specified columns, so we can run aggregation on them. See GroupedData for all the available aggregate functions.

count()

Returns the number of rows in this DataFrame.


In [9]:
data.groupBy("Title").count().sort('count', ascending=False).show()

+--------------------+-----+
|               Title|count|
+--------------------+-----+
| Business Highlights|   37|
|        ALSO IN News|   23|
|After Palestine t...|   23|
|5 Things to Know ...|   19|
|Obama honors Conn...|   17|
|Indonesia leader ...|   16|
|Woman, 4 grandchi...|   15|
|Microsoft's secre...|   14|
|Obama praises Nan...|   13|
|The Latest: Flood...|   13|
|Brexit vote adds ...|   13|
|Prospects still s...|   13|
|Windows 10 releas...|   12|
|Police identify P...|   12|
|Obama wants $4B t...|   12|
|Economy or first ...|   12|
|With HoloLens, Mi...|   12|
|Obama says North ...|   12|
|The Latest: Cruz ...|   11|
|Obama signs bill ...|   11|
+--------------------+-----+
only showing top 20 rows



### Count by "Headline"

In [72]:
data.groupBy("Headline").count().sort('count', ascending=False).show()

+--------------------+-----+
|            Headline|count|
+--------------------+-----+
|"""""""Microsoft ...|   21|
|Read full story f...|   18|
|                null|   15|
|The views express...|   13|
|President Obama o...|   11|
|BETHLEHEM, March ...|   11|
|The court is weig...|   11|
|The family of a P...|   10|
|In 2014, Palestin...|   10|
|Police have ident...|    9|
|Lisa Asberry Davi...|    9|
|President Barack ...|    9|
|The Creighton Eco...|    9|
|Protecting his si...|    8|
|Ironically, Abbas...|    8|
|With fresh fricti...|    8|
|RAMALLAH, March 1...|    8|
|President Obama a...|    8|
|Boris Johnson's P...|    8|
|RAMALLAH, March 3...|    8|
+--------------------+-----+
only showing top 20 rows



### Tokenizer
Tokenization is the process of taking text (such as a sentence) and breaking it into individual terms (usually words). A simple Tokenizer class provides this functionality. The example below shows how to split sentences into sequences of words.

pyspark.sql.functions List of built-in functions available for DataFrame.

In [19]:
import pyspark.sql.functions as F
from pyspark.ml.feature import Tokenizer

split sentences in "Title" into sequences of words

In [54]:
tok_title_wc = Tokenizer(inputCol="Title",outputCol="Tokenizer_Output")
data_tok_title_wc = tok_title_wc.transform(data)
data_tok_title_wc.select("Title","Tokenizer_Output").show()

+--------------------+--------------------+
|               Title|    Tokenizer_Output|
+--------------------+--------------------+
|Obama Lays Wreath...|[obama, lays, wre...|
|A Look at the Hea...|[a, look, at, the...|
|Nouriel Roubini: ...|[nouriel, roubini...|
|Finland GDP Expan...|[finland, gdp, ex...|
|Tourism, govt spe...|[tourism,, govt, ...|
|Intellitec Soluti...|[intellitec, solu...|
| Monday, 29 Feb 2016|[monday,, 29, feb...|
|Obama, stars pay ...|[obama,, stars, p...|
|Fire claims more ...|[fire, claims, mo...|
|Microsoft's new W...|[microsoft's, new...|
|Microsoft Project...|[microsoft, proje...|
|Microsoft sneaks ...|[microsoft, sneak...|
|Greek economy gro...|[greek, economy, ...|
|Big data and the ...|[big, data, and, ...|
|HoloLens dev edit...|[hololens, dev, e...|
|Microsoft Word fo...|[microsoft, word,...|
|Microsoft Band 2 ...|[microsoft, band,...|
|Microsoft prepare...|[microsoft, prepa...|
|Greek economy shr...|[greek, economy, ...|
|Sweden's economy ...|[sweden's,

In [50]:
tok_headline_wc = Tokenizer(inputCol="Headline",outputCol="Tokenizer_Output")
data_tok_headline_wc = tok_headline_wc.transform(data)
data_tok_headline_wc.select("Headline","Tokenizer_Output").show()

+--------------------+--------------------+
|            Headline|    Tokenizer_Output|
+--------------------+--------------------+
|Obama Lays Wreath...|[obama, lays, wre...|
|Tim Haywood, inve...|[tim, haywood,, i...|
|Nouriel Roubini, ...|[nouriel, roubini...|
|Finland's economy...|[finland's, econo...|
|Tourism and publi...|[tourism, and, pu...|
|Over 100 attendee...|[over, 100, atten...|
|RAMALLAH, Februar...|[ramallah,, febru...|
|First lady Michel...|[first, lady, mic...|
|A Hancock County ...|[a, hancock, coun...|
|New Delhi, Feb.29...|[new, delhi,, feb...|
|Microsoft may hav...|[microsoft, may, ...|
|The platform batt...|[the, platform, b...|
|Greece's economy ...|[greece's, econom...|
|Big data analytic...|[big, data, analy...|
|Microsoft’s AR he...|[microsoft’s, ar,...|
|What is A + B? We...|[what, is, a, +, ...|
|The Microsoft Ban...|[the, microsoft, ...|
|It seems that Mic...|[it, seems, that,...|
|Greece's economy ...|[greece's, econom...|
|Sweden's economy ...|[sweden's,

### Wordcount each sentences in "Title"

withColumn(colName, col)

Returns a new DataFrame by adding a column or replacing the existing column that has the same name.

Parameters:	
colName – string, name of the new column.<br>
col – a Column expression for the new column.


pyspark.sql.functions.size(col

Collection function: returns the length of the array or map stored in the column.

pyspark.sql.functions.split(str, pattern

Splits str around pattern (pattern is a regular expression).
<br>Note : pattern is a string represent the regular expression. 

In [45]:
data.withColumn('Wordcount_in_Title', F.size(F.split(F.col('Title'), ' '))).select("Title","Wordcount_in_Title").sort("Wordcount_in_Title",ascending=False).show()

+--------------------+------------------+
|               Title|Wordcount_in_Title|
+--------------------+------------------+
|Microsoft set a n...|                24|
|Obama Hosts His L...|                24|
|Microsoft set a n...|                24|
|Obama Gets RIPPED...|                24|
|Husbands who fly ...|                24|
|Obama Loses Patie...|                23|
|Obama equates glo...|                23|
|'THE BEARS HAVE A...|                23|
|Obama is about to...|                23|
|Dive under the se...|                23|
|The LinkedIn deal...|                23|
|Now Obama tells B...|                23|
|Microsoft set a n...|                23|
|'THE BEARS HAVE A...|                23|
|Satya Nadella on ...|                22|
|This guy saw his ...|                22|
|Obama compares IS...|                22|
|Apple and Microso...|                22|
|'My father had on...|                22|
|Obama meeting wit...|                22|
+--------------------+------------

### Wordcount each sentences in "Headline"

In [48]:
headline_wc = data.withColumn('Wordcount_in_Headline', F.size(F.split(F.col('Headline'), ' '))).select("Headline","Wordcount_in_Headline").sort("Wordcount_in_Headline",ascending=False).show()

+--------------------+---------------------+
|            Headline|Wordcount_in_Headline|
+--------------------+---------------------+
|Osaka, June 2 (IA...|                   81|
|Introduction When...|                   81|
|Microsoft wants y...|                   80|
| Microsoft wants ...|                   80|
| Microsoft wants ...|                   80|
| Microsoft's Saty...|                   79|
|An unwanted softw...|                   79|
|Windows 10 Mobile...|                   78|
|In this Brexit de...|                   78|
|It turns out that...|                   78|
|The Microsoft Sur...|                   78|
|We all know that ...|                   78|
|It's a new year, ...|                   77|
|Submitted by Raul...|                   77|
|President Obama s...|                   77|
|Our economy is ri...|                   77|
|Obama likened Ali...|                   77|
| Microsoft is sup...|                   77|
|Yes, the leaks we...|                   77|
|Obama lik

### Count most frequent words in Title per Topic

In [41]:
data.groupBy(["Title","Topic"]).count().sort("count",ascending=False).show()

+--------------------+---------+-----+
|               Title|    Topic|count|
+--------------------+---------+-----+
| Business Highlights|  economy|   30|
|After Palestine t...|palestine|   23|
|        ALSO IN News|palestine|   23|
|5 Things to Know ...|  economy|   19|
|Obama honors Conn...|    obama|   17|
|Indonesia leader ...|palestine|   16|
|Woman, 4 grandchi...|palestine|   15|
|Brexit vote adds ...|  economy|   13|
|The Latest: Flood...|palestine|   13|
|Microsoft's secre...|microsoft|   13|
|Prospects still s...|  economy|   13|
|With HoloLens, Mi...|microsoft|   12|
|Obama says North ...|    obama|   12|
|Economy or first ...|  economy|   12|
|Police identify P...|palestine|   12|
|The Latest: 6th b...|palestine|   11|
|Revolutionary Gua...|  economy|   11|
|Family of flood v...|palestine|   11|
|President Obama A...|    obama|   11|
|Windows 10 releas...|microsoft|   11|
+--------------------+---------+-----+
only showing top 20 rows



### Count most frequent words in Title per PublishDate/Day

In [44]:
data.groupBy(["Title","PublishDate"]).count().sort("count",ascending=False).show()

+--------------------+-------------------+-----+
|               Title|        PublishDate|count|
+--------------------+-------------------+-----+
|Obama vetoes bill...|2016-01-08 11:07:00|    8|
|EU lawmakers reje...|2016-05-12 07:54:00|    7|
|Obama announces n...|2016-06-20 12:13:00|    6|
|KING 5 is the fir...|2016-02-15 10:07:00|    6|
|If the economy sp...|2016-05-01 09:00:00|    6|
|FEI confirms Qata...|2016-03-24 10:09:00|    6|
|Obama says he'll ...|2016-01-09 09:50:00|    6|
|Messy Brexit coul...|2016-06-17 16:04:00|    6|
|Three years behin...|2016-05-13 14:44:00|    6|
|Microsoft interru...|2016-04-27 15:46:00|    6|
|Obama signs educa...|2015-12-10 11:09:00|    6|
|America beats exp...|2015-12-22 05:30:00|    6|
|Obama says no spe...|2015-12-17 12:19:00|    6|
|Microsoft just bo...|2016-06-13 06:25:00|    6|
|Obama Foundation ...|2015-12-21 08:41:00|    6|
|Obama signs bill ...|2016-05-11 14:30:00|    6|
|Obama: Wall Stree...|2016-03-07 09:44:00|    5|
|Obama just put gu..

### Count most frequent words in Title per Topic

In [63]:
data.groupBy(["Headline","Topic"]).count().sort("count",ascending=False).show()

+--------------------+---------+-----+
|            Headline|    Topic|count|
+--------------------+---------+-----+
|"""""""Microsoft ...|   Forbes|   21|
|Read full story f...|microsoft|   15|
|The views express...|palestine|   13|
|BETHLEHEM, March ...|palestine|   11|
|President Obama o...|    obama|   11|
|The court is weig...|    obama|   11|
|In 2014, Palestin...|palestine|   10|
|The family of a P...|palestine|   10|
|The Creighton Eco...|  economy|    9|
|Lisa Asberry Davi...|palestine|    9|
|Police have ident...|palestine|    9|
|President Barack ...|    obama|    9|
|A Canadian legal ...|palestine|    8|
|I had mixed feeli...|palestine|    8|
|The subject was B...|    obama|    8|
|President Obama a...|    obama|    8|
|RAMALLAH, March 1...|palestine|    8|
|Boris Johnson's P...|palestine|    8|
|RAMALLAH, March 3...|palestine|    8|
|Protecting his si...|    obama|    8|
+--------------------+---------+-----+
only showing top 20 rows



### Count most frequent words in Headline per PublishDate/Day

In [66]:
data.groupBy(["Headline","PublishDate"]).count().sort("count",ascending=False).show()

+--------------------+-------------------+-----+
|            Headline|        PublishDate|count|
+--------------------+-------------------+-----+
|"""""""Microsoft ...|          microsoft|   21|
|Protecting his si...|2016-01-08 11:07:00|    8|
|The Barack Obama ...|2015-12-21 08:41:00|    6|
|WASHINGTON (AP) —...|2016-05-11 14:30:00|    6|
|In an announcemen...|2016-06-13 06:25:00|    6|
|Tired of those Wi...|2016-04-27 15:46:00|    6|
|Despite being ann...|2016-02-15 10:07:00|    6|
|Microsoft is late...|2016-05-13 14:44:00|    6|
|President Barack ...|2016-01-09 09:50:00|    6|
|McLEAN, Va. (AP) ...|2015-12-17 12:19:00|    6|
|LAUSANNE, Switzer...|2016-03-24 10:09:00|    6|
|Spring tidings? T...|2016-05-01 09:00:00|    6|
|The US economy gr...|2015-12-22 05:30:00|    6|
|WASHINGTON (AP) —...|2015-12-10 11:09:00|    6|
|A petition thrown...|2016-06-06 14:47:00|    5|
|The U.K.'s decisi...|2016-06-28 13:20:00|    5|
|MADRID (AP) — Spa...|2016-06-28 02:34:00|    5|
|Microsoft CEO Sat..